<a href="https://colab.research.google.com/github/TriVo131003/OOP2/blob/main/21127456.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq
import time
import random
import psutil
import os
import tracemalloc


def random_chromosome(size):
    return [random.randint(0, size - 1) for _ in range(size)]


def successors(board, n: int):
    new_boards = []
    for i in range(n):
        for j in range(n):
            if j != board[i]:
                new_board = board.copy()
                new_board[i] = j
                new_boards.append(new_board)
    return new_boards


def FitnessFunction(board):
    totalConflict = 0
    n = len(board)
    row = [0] * n
    mainDig = [0] * (n*2)
    subDig = [0] * (n*2)
    for i in range(n):
        index = board[i]
        row[index] += 1
        mainDig[index + i] += 1
        subDig[n - index + i] += 1
    for i in range(n*2):
        if i < n:
            totalConflict += (row[i] * (row[i]-1))/2
        totalConflict += (mainDig[i] * (mainDig[i]-1))/2
        totalConflict += (subDig[i] * (subDig[i]-1))/2
    return int(totalConflict)


def conflicts_count(board):
    totalConflict = 0
    n = len(board)
    row = [0] * n
    mainDig = [0] * (n*2)
    subDig = [0] * (n*2)
    for i in range(n):
        index = board[i]
        row[index] += 1
        mainDig[index + i] += 1
        subDig[n - index + i] += 1
    for i in range(n*2):
        if i < n:
            totalConflict += (row[i] * (row[i]-1))/2
        totalConflict += (mainDig[i] * (mainDig[i]-1))/2
        totalConflict += (subDig[i] * (subDig[i]-1))/2
    return int(totalConflict)


def conflicts_count_opt(pre_board, mov_queen, new_row):
    cnt = 0
    sub = 0
    for i in range(len(pre_board)):
        if i != mov_queen:
            if pre_board[mov_queen] == pre_board[i] or abs(pre_board[mov_queen] - pre_board[i]) == abs(mov_queen - i):
                sub += 1
            if new_row == pre_board[i] or abs(new_row - pre_board[i]) == abs(mov_queen - i):
                cnt += 1
    return cnt - sub


def UCS_Alg(start_state, n):
    frontier = [(conflicts_count(start_state), start_state)]
    explored = set()

    while frontier:
        cost, current_state = heapq.heappop(frontier)

        if cost == 0:
            return current_state

        explored.add(tuple(current_state))

        for successor_state in successors(current_state, n):
            if tuple(successor_state) not in explored:
                heapq.heappush(
                    frontier, (conflicts_count(successor_state), successor_state))

    return None


def Astar_Alg(start_state, n):
    frontier = [(conflicts_count(start_state), 0, start_state)]
    open_list = [frontier]
    explored = set()

    while open_list:
        heuristic, cost, current_state = heapq.heappop(frontier)
        if heuristic == 0:
            return current_state
        explored.add(tuple(current_state))
        for successor_state in successors(current_state, n):
            if tuple(successor_state) not in explored:
                heapq.heappush(
                    frontier, (conflicts_count(successor_state), cost + 1, successor_state))
    return None


def newPopulation(n: int):
    listPopulation = []
    num = random.randint(2, n)
    while num != 0:
        temp = random_chromosome(n)
        heapq.heappush(listPopulation, (FitnessFunction(temp), temp))
        num -= 1
    return listPopulation


def isGoal(population: list):
    for i in population:
        if FitnessFunction(i[1]) == 0:
            return i[1]
    return None


def crossover(x, y):
    n = len(x)
    c = random.randint(0, n - 1)
    return x[0:c] + y[c:n], y[0:c] + x[c:n]


def random_pick(population):
    new_population = []
    n = random.randint(2, len(population))
    for i in range(n):
        new_population.append(population[i][1])
    return new_population


def mutate(x):
    n = len(x)
    c = random.randint(0, n - 1)
    m = random.randint(0, n - 1)
    x[c] = m
    return x


def genetic_Alg(init, n):
    population = newPopulation(n)
    heapq.heappush(population, (FitnessFunction(init), init))
    result = isGoal(population)
    while result == None:
        tempboard = random_pick(population)
        for i in range(0, len(tempboard), 2):
            if i + 2 <= len(tempboard):
                tempboard[i], tempboard[i +
                                        1] = crossover(tempboard[i], tempboard[i+1])
                tempboard[i] = mutate(tempboard[i])
                tempboard[i+1] = mutate(tempboard[i+1])
        for board in tempboard:
            population.pop()
        for board in tempboard:
            heapq.heappush(population, (FitnessFunction(board), board))
        result = isGoal(population)
    return result


def printBoard(board, n):
    result = [['*' for j in range(n)] for i in range(n)]
    for i in range(n):
        result[board[i]][i] = 'Q'
    for i in range(n):
        for j in range(n):
            print(result[i][j], end=' ')
        print()


choice = int(input(
    "1.UCS Algorithm\n2.A* Algorithm\n3.Genetic Algorithm\nYour choice: "))
# Initialize the problem
n = int(input("Input the number of queens: "))

board = random_chromosome(n)
print("inital board: ")
printBoard(board, n)

startTime = time.time()
tracemalloc.start()

if choice == 1:
    board = UCS_Alg(board, n)
elif choice == 2:
    board = Astar_Alg(board, n)
else:
    board = genetic_Alg(board, n)

peak = tracemalloc.get_traced_memory()[1]
tracemalloc.stop()
mem = peak / 1024**2

print("goal board: ")
printBoard(board, n)

print(f"Time cost: {time.time() - startTime:.4f} sec")
print(f"Memory usage: {mem:.2f} MB")


1.UCS Algorithm
2.A* Algorithm
3.Genetic Algorithm
Your choice: 2
Input the number of queens: 100
inital board: 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * Q * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
* * * * * * * * * * * * * * * * * * * * * * * * * * * Q * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
* * * * * * Q * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *